**Deep Learning - Project**      

**Part A-I**                             

Name: Christodoulidis Anastasios         
Α.Μ.: aivc22019

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical

from sklearn.utils import class_weight

In [2]:
# import dataset
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Deep Learning')
partA = pd.read_excel('Dataset2Use_PartA.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Preview Dataset
partA

,365* ( Β.Υ / Κοστ.Πωλ ),Λειτ.Αποτ/Συν.Ενεργ. (ROA),ΧΡΗΜ.ΔΑΠΑΝΕΣ / ΠΩΛΗΣΕΙΣ,ΠΡΑΓΜΑΤΙΚΗ ΡΕΥΣΤΟΤΗΤΑ : (ΚΕ-ΑΠΟΘΕΜΑΤΑ) / Β.Υ,(ΑΠΑΙΤ.*365) / ΠΩΛ.,Συν.Υποχρ/Συν.Ενεργ,Διάρκεια Παραμονής Αποθεμάτων,Λογαριθμος Προσωπικού,ΕΝΔΕΙΞΗ ΕΞΑΓΩΓΩΝ,ΕΝΔΕΙΞΗ ΕΙΣΑΓΩΓΩΝ,ΕΝΔΕΙΞΗ ΑΝΤΙΠΡΟΣΩΠΕΙΩΝ,ΕΝΔΕΙΞΗ ΑΣΥΝΕΠΕΙΑΣ (=2) (ν+1),ΕΤΟΣ
0,1014.128862,0.011751,0.038505,0.965479,567.866550,0.532258,545.195964,0.301030,0,1,0,1,2006
1,561.974631,0.099662,0.015775,1.415467,335.917481,0.660556,7.950910,1.176091,1,1,1,1,2006
2,826.919025,0.009594,0.093562,1.575034,350.618086,0.550912,28.602221,1.204120,0,1,1,1,2006
3,493.176956,0.196284,0.000581,1.346780,414.508709,0.683717,1.216888,1.431364,0,0,0,1,2006
4,100.421465,0.179038,0.000173,3.299687,118.548381,0.254496,52.346321,1.518514,0,1,1,1,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10711,129.571104,0.046147,0.000000,1.260416,0.380702,0.726776,1.724963,0.000000,1,0,0,2,2009
10712,1112.742613,0.032630,0.134262,0.664461,1.319013,1.151101,12.379601,0.000000,0,0,0,2,2009
10713,1194.250878,-0.172327,0.200000,0.572939,1.872967,1.030767,422.327531,1.146128,0,1,0,2,2009
10714,1500.000000,-0.073754,0.200000,0.715888,16.158480,1.000224,1290.137484,1.079181,1,1,0,2,2009


In [4]:
# Dataset Dimensions
partA.shape

(10716, 13)

In [5]:
# Drop column "ΕΤΟΣ"
partA = partA.drop('ΕΤΟΣ', axis=1)
partA.shape

(10716, 12)

In [6]:
# split I/O
x = partA.iloc[:, :-1]
y = partA.iloc[:, -1]

In [7]:
# Class Counts
print('Class Counts: ')
print(y.value_counts())

Class Counts: 
1    10468
2      248
Name: ΕΝΔΕΙΞΗ ΑΣΥΝΕΠΕΙΑΣ (=2) (ν+1), dtype: int64


In [8]:
# Data Normalization

# Get the column names
column_names = x.columns.tolist()

# Create an instance of StandardScaler
scaler = MinMaxScaler()

# Perform normalization
x = scaler.fit_transform(x)

# Create DataFrame with the normalized data
x = pd.DataFrame(x, columns=column_names)

In [9]:
# split train & test sets(20%)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [10]:
# Class counts in train set
y_train.value_counts()

1    8372
2     200
Name: ΕΝΔΕΙΞΗ ΑΣΥΝΕΠΕΙΑΣ (=2) (ν+1), dtype: int64

In [11]:
# Class Counts in test set
y_test.value_counts()

1    2096
2      48
Name: ΕΝΔΕΙΞΗ ΑΣΥΝΕΠΕΙΑΣ (=2) (ν+1), dtype: int64

In [12]:
# Create Results Dataframe
results = pd.DataFrame(columns=['Classifier Name', 'Training or Test Set', 'Number of Training Samples',
                           'Number of Non-Healthy Companies in Training Sample', 'TP', 'TN', 'FP', 'FN',
                           'Precision', 'Recall', 'F1 Score', 'Accuracy'])

In [13]:
# Calculate class weights (for LogReg, DT, SVM)
class_weights = class_weight.compute_class_weight('balanced', classes=[1, 2], y=y_train)
class_weights

array([ 0.51194458, 21.43      ])

In [14]:
# Change class numbers to -> Healthy Company=0, Bankrupt=1
y_train = y_train-1
y_test = y_test-1

In [15]:
def fit_and_predict(classifier_name, train_or_test, x_train, y_train, x_test=None):

  """
  Given the 'classifier_name' and
  the type of set we want to predict 'Training' or 'Test',
  fit the model according to x_train & y_train,
  predict the output for x_train or x_test using the fitted model and
  return the predicted output.

  """

  classifiers = {
      'lda': LinearDiscriminantAnalysis(),
      'logreg': LogisticRegression(),
      'logregw': LogisticRegression(class_weight={0: class_weights[0], 1: class_weights[1]}),
      'dt': DecisionTreeClassifier(),
      'dtw': DecisionTreeClassifier(class_weight={0: class_weights[0], 1: class_weights[1]}),
      'kn': KNeighborsClassifier(n_neighbors=5),
      'nb': GaussianNB(),
      'svm': SVC(kernel='rbf', C=1.0, gamma='scale'),
      'svmw': SVC(class_weight={0: class_weights[0], 1: class_weights[1]})
      }

  if classifier_name not in classifiers:
    raise ValueError("Invalid classifier name specified.")

  # Fit model
  classifier = classifiers[classifier_name]
  classifier.fit(x_train, y_train)

  # Predict output
  if train_or_test=='Training':
    y_pred = classifier.predict(x_train)
  elif train_or_test=='Test':
    y_pred = classifier.predict(x_test)

  return y_pred

In [16]:
def calculate_results(classifier_name, dataset_type, no_of_tr_samples, no_of_non_healthy_companies_in_tr_set, true, pred):

  """
  Given the 'classifier_name', 'dataset_type', 'no_of_tr_samples', 'no_of_non_healthy_companies_in_tr_set',
  'true' output and 'pred' output, calculate TP,FP,FN,TN,precision,recall,f1-score and accuracy of the model
  and add them to the results dataframe.
  """

  # Calculate true positives, true negatives, false positives, false negatives
  confusion_mat = confusion_matrix(true, pred)
  tp, fn, fp, tn = confusion_mat.ravel()

  precision = round(tp/(tp+fp),2)
  recall = round(tp/(tp+fn),2)
  f1 = round((2*precision*recall) / (precision+recall),2)
  accuracy = round((tp+tn)/(tp+tn+fp+fn),2)
  accuracy = f'Total: {accuracy}, C1: {round(tp/len(true[true==0]),2)}, C2: {round(tn/len(true[true==1]),2)}'

  result=[classifier_name, dataset_type, no_of_tr_samples,
          no_of_non_healthy_companies_in_tr_set, tp, tn, fp, fn, precision, recall, f1, accuracy]

  results.loc[len(results)] = result

In [17]:
# Linear Discriminant Analysis
y_pred = fit_and_predict('lda','Training', x_train, y_train)
calculate_results('LinearDiscriminantAnalysis', 'Training', len(x_train), len(y_train[y_train==1]), y_train, y_pred)

y_pred = fit_and_predict('lda','Test', x_train, y_train, x_test)
calculate_results('LinearDiscriminantAnalysis', 'Test', len(x_train), len(y_train[y_train==1]), y_test, y_pred)

In [18]:
# Logistic Regression
y_pred = fit_and_predict('logreg','Training', x_train, y_train)
calculate_results('LogisticRegression', 'Training', len(x_train), len(y_train[y_train==1]), y_train, y_pred)

y_pred = fit_and_predict('logreg','Test', x_train, y_train, x_test)
calculate_results('LogisticRegression', 'Test', len(x_train), len(y_train[y_train==1]), y_test, y_pred)

In [19]:
# Logistic Regression with weights
y_pred = fit_and_predict('logregw','Training', x_train, y_train)
calculate_results('LogisticRegression-weighted', 'Training', len(x_train), len(y_train[y_train==1]), y_train, y_pred)

y_pred = fit_and_predict('logregw','Test', x_train, y_train, x_test)
calculate_results('LogisticRegression-weighted', 'Test', len(x_train), len(y_train[y_train==1]), y_test, y_pred)

In [20]:
# Desicion Trees
y_pred = fit_and_predict('dt','Training', x_train, y_train)
calculate_results('DecisionTree', 'Training', len(x_train), len(y_train[y_train==1]), y_train, y_pred)

y_pred = fit_and_predict('dt','Test', x_train, y_train, x_test)
calculate_results('DecisionTree', 'Test', len(x_train), len(y_train[y_train==1]), y_test, y_pred)

In [21]:
# Desicion Trees with weights
y_pred = fit_and_predict('dtw','Training', x_train, y_train)
calculate_results('DecisionTree-weighted', 'Training', len(x_train), len(y_train[y_train==1]), y_train, y_pred)

y_pred = fit_and_predict('dtw','Test', x_train, y_train, x_test)
calculate_results('DecisionTree-weighted', 'Test', len(x_train), len(y_train[y_train==1]), y_test, y_pred)

In [22]:
# k-Nearest Neighbors
y_pred = fit_and_predict('kn','Training', x_train, y_train)
calculate_results('KNeighbors', 'Training', len(x_train), len(y_train[y_train==1]), y_train, y_pred)

y_pred = fit_and_predict('kn','Test', x_train, y_train, x_test)
calculate_results('KNeighbors', 'Test', len(x_train), len(y_train[y_train==1]), y_test, y_pred)

In [23]:
# Naïve Bayes
y_pred = fit_and_predict('nb','Training', x_train, y_train)
calculate_results('NaïveBayes', 'Training', len(x_train), len(y_train[y_train==1]), y_train, y_pred)

y_pred = fit_and_predict('nb','Test', x_train, y_train, x_test)
calculate_results('NaïveBayes', 'Test', len(x_train), len(y_train[y_train==1]), y_test, y_pred)

In [24]:
#  Support Vector Machines
y_pred = fit_and_predict('svm','Training', x_train, y_train)
calculate_results('SupportVectorMachines', 'Training', len(x_train), len(y_train[y_train==1]), y_train, y_pred)

y_pred = fit_and_predict('svm','Test', x_train, y_train, x_test)
calculate_results('SupportVectorMachines', 'Test', len(x_train), len(y_train[y_train==1]), y_test, y_pred)

In [25]:
#  Support Vector Machines with weights
y_pred = fit_and_predict('svmw','Training', x_train, y_train)
calculate_results('SupportVectorMachines-weighted', 'Training', len(x_train), len(y_train[y_train==1]), y_train, y_pred)

y_pred = fit_and_predict('svmw','Test', x_train, y_train, x_test)
calculate_results('SupportVectorMachines-weighted', 'Test', len(x_train), len(y_train[y_train==1]), y_test, y_pred)

In [26]:
# Neural Network

# Create Model
model = keras.models.Sequential()
model.add(keras.layers.Dense(9, input_dim=x_train.shape[1], activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))

#display the architecture
model.summary()

#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy',
metrics=['accuracy'])

num_classes = 2
y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes)

#train the model
model.fit(x_train, y_train_one_hot,epochs=100, verbose=False)

# Predict classes for training and test data
y_pred_train = np.argmax(model.predict(x_train), axis=1)
y_pred_test = np.argmax(model.predict(x_test), axis=1)

calculate_results('NeuralNetwork', 'Training', len(x_train), len(y_train[y_train==1]), y_train, y_pred_train)
calculate_results('NeuralNetwork', 'Test', len(x_train), len(y_train[y_train==1]), y_test, y_pred_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 108       
                                                                 
 dense_1 (Dense)             (None, 2)                 20        
                                                                 
Total params: 128
Trainable params: 128
Non-trainable params: 0
_________________________________________________________________
67/67 [==============================] - 0s 1ms/step


In [27]:
# Neural Network - weighted

# Define the class weights dictionary
class_weights = {0: class_weights[0], 1: class_weights[1]}

model = keras.models.Sequential()
model.add(keras.layers.Dense(9, input_dim=x_train.shape[1], activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))

#display the architecture
model.summary()

#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy',
metrics=['accuracy'])

# num_classes = 2
# y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes)

#train the model
model.fit(x_train, y_train_one_hot,class_weight=class_weights, epochs=100, verbose=False)

# Predict classes for training and test data
y_pred_train = np.argmax(model.predict(x_train), axis=1)
y_pred_test = np.argmax(model.predict(x_test), axis=1)

calculate_results('NeuralNetwork-weighted', 'Training', len(x_train), len(y_train[y_train==1]), y_train, y_pred_train)
calculate_results('NeuralNetwork-weighted', 'Test', len(x_train), len(y_train[y_train==1]), y_test, y_pred_test)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 9)                 108       
                                                                 
 dense_3 (Dense)             (None, 2)                 20        
                                                                 
Total params: 128
Trainable params: 128
Non-trainable params: 0
_________________________________________________________________
67/67 [==============================] - 0s 1ms/step


In [29]:
results

,Classifier Name,Training or Test Set,Number of Training Samples,Number of Non-Healthy Companies in Training Sample,TP,TN,FP,FN,Precision,Recall,F1 Score,Accuracy
0,LinearDiscriminantAnalysis,Training,8572,200,8281,14,186,91,0.98,0.99,0.98,"Total: 0.97, C1: 0.99, C2: 0.07"
1,LinearDiscriminantAnalysis,Test,8572,200,2080,2,46,16,0.98,0.99,0.98,"Total: 0.97, C1: 0.99, C2: 0.04"
2,LogisticRegression,Training,8572,200,8370,1,199,2,0.98,1.00,0.99,"Total: 0.98, C1: 1.0, C2: 0.0"
3,LogisticRegression,Test,8572,200,2096,0,48,0,0.98,1.00,0.99,"Total: 0.98, C1: 1.0, C2: 0.0"
4,LogisticRegression-weighted,Training,8572,200,6484,156,44,1888,0.99,0.77,0.87,"Total: 0.77, C1: 0.77, C2: 0.78"
5,LogisticRegression-weighted,Test,8572,200,1627,35,13,469,0.99,0.78,0.87,"Total: 0.78, C1: 0.78, C2: 0.73"
6,DecisionTree,Training,8572,200,8372,200,0,0,1.00,1.00,1.00,"Total: 1.0, C1: 1.0, C2: 1.0"
7,DecisionTree,Test,8572,200,2052,4,44,44,0.98,0.98,0.98,"Total: 0.96, C1: 0.98, C2: 0.08"
8,DecisionTree-weighted,Training,8572,200,8372,200,0,0,1.00,1.00,1.00,"Total: 1.0, C1: 1.0, C2: 1.0"
9,DecisionTree-weighted,Test,8572,200,2058,6,42,38,0.98,0.98,0.98,"Total: 0.96, C1: 0.98, C2: 0.12"
